## Computing expected consumption

Assume consumption is given by $C(\epsilon)=U(e^{\epsilon})$ with $\epsilon$ is a random variable with standard deviation ${\sigma}_{\epsilon}=0.05$.

Set utility of consumption to be: $U(x)=\frac{x^{-\gamma}}{-\gamma}$.

Goal is to compare several ways to compute $E_{\epsilon}(C(\epsilon))$ .

We'll use the calibration $\gamma=40$ and ${\sigma}_{\epsilon}=0.05$. 

__(optional) Compute the result in closed form__

Hint: $$\int_{-\infty}^{\infty}e^{-a x^2+bx +c} dx = \sqrt{\frac{\pi}{a}} e^{\frac{b^2}{4a}+c}$$

__Compute the result by approximating an integral with `QuadGK`__

__Compute the result using Monte-Carlo simulations. Evaluate the variance of the estimation.__

__Compute the result using Gauss-Hermite quadrature__

## Cournot model

In Cournot competition, and oligopolistic firm equates marginal cost to marginal
revenue: $p + q dp/dq$.

Pricing decision will depend on the demand elasticity: $$dp/dq = 1/D'(p)$$

Our goal is to find the effective supply function $S(p)$.

It satisfies the functional equation:
    
$$p + S(p)/D'(p) - MC(S(p)) = 0$$

Assume 
    
$$D(p) = p^{-\eta}$$
$$MC(q) = \alpha \sqrt{q} + q^2$$

Use the calibration $\alpha=1$, $\eta=1.5$.


__Direct approach:__

- write a routine which solves $S(p)$ for given $p$
- write a routine which solves $S(p)$ for a vector of $p$

__Collocation__

- construct a Chebychev functional space using BasisMatrices, with matching chebychev nodes
- inspect the Basis matrix and plot the various basis polynomials
- solve the above problem with collocation
- plot the result